<b> Name : Deepak Gautam <br />
NetID: dg1308 </b>

## Neo4J

Neo4j is an open-source graph database implemented in Java. The developers describe Neo4j as "embedded, disk-based, fully transactional Java persistence engine that stores data structured in graphs rather than in tables". Neo4j is the most popular graph database.

Manual and Installation: http://neo4j.com/developer/get-started/

Neo4J installation Ubuntu:

```
echo 'deb http://debian.neo4j.org/repo stable/' > /etc/apt/sources.list.d/neo4j.list
echo 'deb http://debian.neo4j.org/repo stable/' | sudo tee -a /etc/apt/sources.list.d/neo4j.list
sudo apt-get update
sudo apt-get install neo4j
```

Python Clients:

**neo4jrestclient**: The first objective of Neo4j Python REST Client is to make transparent for Python programmers the use of a local database through python-embedded or a remote database thanks to Neo4j REST Server. So, the syntax of this API is fully compatible with python-embedded. However, a new syntax is introduced in order to reach a more pythonic style.

`sudo pip install neo4jrestclient`

**Py2neo**: Py2neo is a client library and comprehensive toolkit for working with Neo4j from within Python applications and from the command line. The core library has no external dependencies and has been carefully designed to be easy and intuitive to use.

`sudo pip install py2neo`


The simplest way to try out a connection to the Neo4j server is via the console. Once you have started a local Neo4j server, open a new Python console and enter the following:

In [17]:
from py2neo import Graph, authenticate
graph = Graph()

In [18]:
authenticate("localhost:7474", "neo4j", "")
graph = Graph("http://localhost:7474/db/data/")

#### Nodes & Relationships

Nodes and relationships are the fundamental building blocks of a Neo4j graph and both have a corresponding class in py2neo. Assuming we’ve already established a connection to the server (as above) let’s build a simple graph with two nodes and one relationship

A graph node that may optionally be bound to a remote counterpart in a Neo4j database. Nodes may contain a set of named properties and may have one or more labels applied to them: (http://py2neo.org/2.0/essentials.html#nodes)

In [19]:
from py2neo import Node, Relationship, Path
alice = Node("Person", name="Alice")
bob = Node("Person", name="Bob")

In [20]:
alice.properties["name"]

u'Alice'

In [21]:
alice.labels

{'Person'}

In [22]:
alice.labels.add("Employee")
alice.properties["employee_no"] = 3456
alice

<Node labels=set(['Employee', 'Person']) properties={'name': u'Alice', 'employee_no': 3456}>

A graph relationship that may optionally be bound to a remote counterpart in a Neo4j database. Relationships require a triple of start node, relationship type and end node and may also optionally be given one or more properties: (http://py2neo.org/2.0/essentials.html#relationships)

In [23]:
alice_knows_bob = Relationship(alice, "KNOWS", bob, since=1999)
graph.create(alice_knows_bob)

(<Relationship graph=u'http://localhost:7474/db/data/' ref=u'relationship/0' start=u'node/0' end=u'node/1' type=u'KNOWS' properties={'since': 1999}>,)

In [24]:
graph.create(alice_knows_bob)

(<Relationship graph=u'http://localhost:7474/db/data/' ref=u'relationship/0' start=u'node/0' end=u'node/1' type=u'KNOWS' properties={'since': 1999}>,)

When first created, `Node` and `Relationship` objects exist only in the client; nothing has yet been written to the server. The `Graph.create` method shown above creates corresponding server-side objects and automatically binds each local object to its remote counterpart. Within py2neo, binding is the process of applying a URI to a client object thereby allowing future client-server synchonisation operations to occur.

#### Pushing & Pulling

Client-server communication over `REST` can be chatty if not used carefully. Whenever possible, py2neo attempts to minimise the amount of chatter between the client and the server by batching and lazily retrieving data. Most read and write operations are explicit, allowing the Python application developer a high degree of control over network traffic.

To illustrate synchronisation, let’s give Alice and Bob an age property each. Longhand, this is done as follows:

In [25]:
alice.properties["age"] = 33
bob.properties["age"] = 44
alice.push()
bob.push()

Here, we add a new property to each of the two local nodes and `push` the changes in turn. This results in two separate `HTTP` calls being made to the server which can be seen more clearly with the debugging function, `watch`:

In [26]:
from py2neo import watch
# watch("httpstream")
alice.push()

In [27]:
bob.push()

To squash these two separate push operations into one, we can use the Graph.push method instead:

In [28]:
graph.push(alice, bob)

#### Cypher

**Single Statements**

Neo4j has a built-in data query and manipulation language called Cypher. To execute Cypher from within py2neo, simply use the cypher attribute of a Graph instance and call the execute method:

In [29]:
graph.cypher.execute("CREATE (c:Person {name:{N}}) RETURN c", {"N": "Carol"})

   | c                         
---+----------------------------
 1 | (n2:Person {name:"Carol"})

The object returned from this call is a RecordList which is rendered by default as a table of results. Each item in this list is a Record instance:

In [30]:
for record in graph.cypher.execute("CREATE (d:Person {name:'Dave'}) RETURN d"):
    print(record)

 d                        
---------------------------
 (n3:Person {name:"Dave"})



A Record exposes its values through both named attributes and numeric indexes. Therefore, if a Cypher query returns a column called name, that column can be accessed through the record attribute called name:

In [31]:
for record in graph.cypher.execute("MATCH (p:Person) RETURN p.name AS name"):
    print(record.name)

Alice
Bob
Carol
Dave


#### Transactions

Neo4j 2.0 extended the REST interface to allow multiple Cypher statements to be sent to the server as part of a single transaction. Transactions such as these allow far more control over the logical grouping of statements and can also offer vastly better performance compared to individual statements by submitting multiple statements in a single HTTP request.

To use this endpoint, firstly call the begin method on the Graph.cypher resource to create a transaction, then use the methods listed below on the new CypherTransaction object:

**append(statement, [parameters])** - add a statement to the queue of statements to be executed (this does not pass any statements to the server)

**process()** - push all queued statements to the server for execution and return the results from those statements

**commit()** - push all queued statements to the server for execution and commit the transaction (returns the results from all queued statements)

**rollback()** - roll the transaction back


In [32]:
tx = graph.cypher.begin()
statement = "MATCH (a {name:{A}}), (b {name:{B}}) CREATE (a)-[:KNOWS]->(b)"
for person_a, person_b in [("Alice", "Bob"), ("Bob", "Dave"), ("Alice", "Carol")]:
    tx.append(statement, {"A": person_a, "B": person_b})

tx.commit()

#### Unique Nodes

Many applications require some form of uniqueness to be maintained for the data they manage. Neo4j’s optional schema feature allows such uniqueness constraints to be applied to a graph based on a combination of label and property and py2neo exposes this capability through the create_uniqueness_constraint method of the `Graph.schema` attribute:

In [33]:
graph.schema.create_uniqueness_constraint("Person", "email")

If an attempt is made to create two nodes with similar unique property values, an exception will be raised and no new node will be created. To `get` or `create` a node with a particular label and property, the `merge_one` method can be used instead:

In [34]:
xavier = graph.merge_one("Person", "email", "charles@x.men")

#### Unique Paths

When it comes to building unique relationships, the `Graph.create_unique` method is a handy wrapper for the Cypher `CREATE UNIQUE` clause. This method can accept one or more Path objects, including Relationship objects (which are simply a subclass of Path).

Let’s assume we want to pick up two nodes based on their email address properties and ensure they are connected by a KNOWS relationship:

In [35]:
alice = graph.merge_one("Person", "email", "alice@example.com")
bob = graph.merge_one("Person", "email", "bob@email.net")
graph.create_unique(Relationship(alice, "KNOWS", bob))

(<Relationship graph=u'http://localhost:7474/db/data/' ref=u'relationship/4' start=u'node/5' end=u'node/6' type=u'KNOWS' properties={}>,)

We could of course extend this to create a unique chain of relationships:

In [36]:
carol = graph.merge_one("Person", "email", "carol@foo.us")
dave = graph.merge_one("Person", "email", "dave@dave.co.uk")
graph.create_unique(Path(alice, "KNOWS", bob, "KNOWS", carol, "KNOWS", dave))

(<Path graph=u'http://localhost:7474/db/data/' start=u'node/5' end=u'node/8' order=4 size=3>,)

#### Delete a Graph

`graph.delete_all()`

To permanently delete a graph

shutdown neo4j server

`sudo service neo4j-service stop`

remove all files from database 

`sudo rm -rf data/*`

restart the server

`sudo service neo4j-service start`

In [37]:
graph.delete_all()